In [ ]:
using Pkg;
pkg"registry add https://github.com/FrameFunVC/FrameFunRegistry"

In [ ]:
using Pkg;Pkg.activate(pwd())
Pkg.instantiate()
apath=normpath(joinpath(pwd(),"."));
width = ".5\\textwidth"
height = ".25\\textwidth"

# 1D example figure

In [ ]:
using BasisFunctions, DomainSets, PGFPlotsX, FrameFun, LaTeXStrings, DocumentPGFPlots

In [ ]:
N = 31
f = identity
P = ExtensionFramePlatform(platform(Fourier(10)→(-2.0..2.0)),-1.0..1.0)
F = Expansion(basis(dictionary(P,N)),coefficients(Fun(f, P, N)))
t = PeriodicEquispacedGrid(2000,-4,4)
tt = PeriodicEquispacedGrid(1000,-2,2)
P1Dexampleframe = @pgf Axis(
    {width=width,height=height,ymin=-2.5,ymax=2.5},
    PlotInc({mark="none", style="dashed,blue"},Table([t, circshift(repeat(real.((F)(tt)),2),500)])),
    PlotInc({mark="none",style="blue,thick"},Table([tt, real.((F)(tt))])),
    PlotInc({mark="none",style="red, thick"},Table([t, t]))
    )

In [ ]:
t = PeriodicEquispacedGrid(2000,-4,4)
tt = PeriodicEquispacedGrid(500,-1,1)
P = platform(Fourier(10,-1,1))
F = Fun(f, P, N)
P1Dexamplebasis = @pgf Axis(
    {width=width,height=height,ymin=-2.5,ymax=2.5},
    PlotInc({mark="none", style="dashed,blue"},Table([t, circshift(repeat(real.((F)(tt)),4),250)])),
    PlotInc({mark="none",style="blue,thick"},Table([tt, real.((F)(tt))])),
    PlotInc({mark="none",style="red, thick"},Table([t, t]))
    )

In [ ]:
# DocumentPGFPlots.savefigs(joinpath(apath,"img","1Dexample"),P1Dexampleframe)
# DocumentPGFPlots.savefigs(joinpath(apath,"img","1Dexample_Gibbs"),P1Dexamplebasis)

# Unexpected problem figure

In [ ]:
using BasisFunctions, DomainSets, PGFPlotsX, FrameFun, LaTeXStrings, DocumentPGFPlots

In [ ]:
f = exp
N1 = 20
N2 = 40
Pbasis = platform(ChebyshevT(1,-1,1)) 
P = ExtensionFramePlatform(platform(ChebyshevT(1,-2.,2.)), -1.0..1.0)

Fbasis = Fun(f, Pbasis, N2)
Fbasissmall = Fun(f, Pbasis, N1)

Fframe = Fun(f, P, N2;solverstyle=DirectStyle(),directsolver=:qr)
Fframesmall = Fun(f, P, N1;solverstyle=DirectStyle(),directsolver=:qr)

In [ ]:
Pbasis = @pgf Axis({width=width,height=height,ymode="log",ymin=1e-18,ymax=4,ylabel=L"$|c_k|$",xlabel=L"$k$"},
PlotInc({},Table([1:N2,abs.(coefficients(Fbasis))])),
PlotInc({},Table([1:N1, abs.(coefficients(Fbasissmall))]))
    )

In [ ]:
Pframe = @pgf Axis({width=width,height=height,ymode="log",ymin=1e-18,ymax=4,ylabel=L"$|c_k|$",xlabel=L"$k$"},
PlotInc({},Table([1:N2,abs.(coefficients(Fframe))])),
PlotInc({},Table([1:N1,abs.(coefficients(Fframesmall))]))
    )

In [ ]:
# DocumentPGFPlots.savefigs(joinpath(apath,"img","coefsinbasis"),Pbasis)
# DocumentPGFPlots.savefigs(joinpath(apath,"img","coefsinframe"),Pframe)

In [ ]:
f = exp
err1 = zeros(N2)
err2 = similar(err1)
for (i,N) in enumerate(1:N2)
    Pbasis = platform(ChebyshevT(1,-1,1)) 
    P = ExtensionFramePlatform(platform(ChebyshevT(1,-2.,2.)), -1.0..1.0)

    Fbasis = Fun(f, Pbasis, N;threshold=1e-14,normalizedsampling=true)
    Fframe = Fun(f, P, N;solverstyle=DirectStyle(),threshold=1e-14,normalizedsampling=true)
    err1[i] = maxerror(f, Fbasis)
    err2[i] = maxerror(f, Fframe)
end


In [ ]:
Pbasis = @pgf Axis({width=width,height=height,ymode="log",ymin=1e-18,ymax=4,ylabel=L"$\|f-f_N\|_{\infty}$",xlabel=L"$N$"},
PlotInc({},Table([1:N2,err1]))
    )

In [ ]:
Pframe = @pgf Axis({width=width,height=height,ymode="log",ymin=1e-18,ymax=4,ylabel=L"$\|f-f_N\|_{\infty}$",xlabel=L"$N$"},
PlotInc({},Table([1:N2,err2]))
    )

In [ ]:
# DocumentPGFPlots.savefigs(joinpath(apath,"img","errinbasis"),Pbasis)
# DocumentPGFPlots.savefigs(joinpath(apath,"img","errinframe"),Pframe)

# Smoothing figure

In [ ]:
using BasisFunctions, DomainSets, PGFPlotsX, FrameFun, LaTeXStrings, DocumentPGFPlots

In [ ]:
width = ".5\\textwidth"
height = ".3\\textwidth"

In [ ]:
P = ExtensionFramePlatform(platform(ChebyshevT(1) → (-2.0..2.)), -1.0..1.0)
f = exp

N  = 61
fscale = (d,i) -> 10.0^-4+abs(i)+abs(i)^2+abs(i)^3
Frough_coef = coefficients(Fun(f, P, N;threshold=1e-14,normalizedsampling=true,solverstyle=DirectStyle()))
Fsmooth_coef = coefficients(Fun(f, P, N;threshold=1e-14,normalizedsampling=true,solverstyle=DirectStyle(),directsolver=:qr,weightedAZ=true,AZ_Cweight=sobolevAZweight(AZ_A(P,N); scaling=fscale)))
Fwa_coef = Fun(f,P,threshold=1e-14,normalizedsampling=true,δ=0,p0=4,adaptivestyle=:greedy,maxlength=N-1,solverstyle=DirectStyle(),directsolver=:qr,weightedAZ=true,maxiterations=N)
Frough = Expansion(basis(dictionary(P,N)), Frough_coef)
Fsmooth = Expansion(basis(dictionary(P,N)), Fsmooth_coef)
Fwa = Expansion(basis(dictionary(P,N)), Fwa_coef)

N2 = 101
Frough_coef2 = coefficients(Fun(f, P, N2;threshold=1e-14,normalizedsampling=true,solverstyle=DirectStyle(),directsolver=:qr))
Fsmooth_coef2 = coefficients(Fun(f, P, N2;threshold=1e-14,normalizedsampling=true,solverstyle=DirectStyle(),directsolver=:qr,weightedAZ=true,AZ_Cweight=sobolevAZweight(AZ_A(P,N2); scaling=fscale)))
Fwa_coef2 = Fun(f,P,threshold=1e-14,normalizedsampling=true,δ=0,p0=4,adaptivestyle=:greedy,maxlength=N2-1,weightedAZ=true,maxiterations=N2,solverstyle=DirectStyle(),directsolver=:qr)
Frough2 = Expansion(basis(dictionary(P,N2)), Frough_coef2)
Fsmooth2 = Expansion(basis(dictionary(P,N2)), Fsmooth_coef2)
Fwa2 = Expansion(basis(dictionary(P,N2)), Fwa_coef2)

In [ ]:
s1 = @pgf {style="red",mark="*",mark_options="red"}
s2 = @pgf {style="blue",mark="square*",mark_options="blue"}
s3 = @pgf {style="green",mark="diamond*",mark_options="green"}
s4 = @pgf {style="brown",mark="diamond"}

s1d = @pgf {style="red,dashed",mark="o"}
s2d = @pgf {style="blue,dashed",mark="square"}
s3d = @pgf {style="green,dashed",mark="diamond"}
s4d = @pgf {style="brown,dashed",mark="diamond"}

In [ ]:
t = LinRange(-2,2,1000)
P1 = @pgf Axis(
    {xlabel = L"$t$",#ylabel=L"$\mathcal P^\epsilon_{M,N}f(t)$",
    legend_cell_align="left",
    width=width,height=height,ymin=-5,ymax=5,legend_pos="south west"},
    PlotInc({mark="none",style="very thick",color="red"},Table([t, real((Frough).(t))])),
    PlotInc({mark="none",style="very thick",color="blue"},Table([t, real.((Fsmooth).(t))])),
    PlotInc({mark="none",style="very thick",color="green"},Table([t, real.((Fwa).(t))])),
    )

In [ ]:
P2 = @pgf Axis(
    {xlabel=L"$t$",#ylabel=L"$|f(x)-(\mathcal P^\epsilon_{M,N}f)(x)|$",
    width=width,height=height,ymode="log"},
    PlotInc({mark="none",style="very thick",color="red"},Table([t, eps() .+ abs.(Frough.(t)-f.(t))])),
    PlotInc({mark="none",style="very thick",color="blue"},Table([t, eps() .+ abs.(Fsmooth.(t)-f.(t))])),
    PlotInc({mark="none",style="very thick",color="green"},Table([t, eps() .+ abs.(Fwa.(t)-f.(t))])),
)

In [ ]:
r =sortperm(map(x->convert(Int,x),ordering(dictionary(P,N))));
r2 =sortperm(map(x->convert(Int,x),ordering(dictionary(P,N2))));

In [ ]:
P3 = @pgf Axis(
    {xlabel=L"$k$",#ylabel=L"$|c_k|$",
    width=width,height=height,ymode="log",
        legend_pos="south west",legend_style={legend_columns=-1,fill_opacity=0.6,text_opacity=1},
    legend_cell_align={left}},
    PlotInc(s1, Table([1:N, 
                abs.(Frough_coef)])),
    PlotInc(s2, Table([1:N, 
                abs.(Fsmooth_coef)])),
    PlotInc(s3, Table([1:N, 
                abs.(Fwa_coef)])),
    
    PlotInc(s1d, Table([1:N2, 
                abs.(Frough_coef2)])),
    PlotInc(s2d, Table([1:N2, 
                abs.(Fsmooth_coef2)])),
    PlotInc(s3d, Table([1:N2, 
                abs.(Fwa_coef2)])),
)

In [ ]:
# DocumentPGFPlots.savefigs(joinpath(apath,"img","approximation"),P1)
# DocumentPGFPlots.savefigs(joinpath(apath,"img","error"),P2)
# DocumentPGFPlots.savefigs(joinpath(apath,"img","coefficients"),P3)

# Non decrease of L2 figure

In [ ]:
using BasisFunctions, DomainSets, PGFPlotsX, FrameFun, LaTeXStrings, DocumentPGFPlots
width = ".5\\textwidth"
height = ".3\\textwidth"

### Constant function

In [ ]:
f = x->1.
m_mon = 30
err_mon = zeros(m_mon)
csize_mon = zeros(m_mon)
tol = 1e-14; cutoff = 1e-5
for N in 1:m_mon
    P = ExtensionFramePlatform(platform(Fourier(1,-2,2)),-1.0..1.0)
    A = Fun(f, P, N;samplingstyle=GramStyle(),directsolver=regularized_SVD_solver, 
        threshold=cutoff, discrete=false, rtol=tol, atol=tol)
    err_mon[N] = L2error(f,A,  atol=tol, rtol=tol)
    csize_mon[N] = norm(coefficients(A))
end

In [ ]:
upper_mon = [NaN, (err_mon[2:end].+sqrt(cutoff)*csize_mon[2:end])...];

In [ ]:
Pnonmonotone = @pgf Axis(
    {width=width,height=height,ymode="log",xlabel=L"$N$", 
    cycle_list_name="mark list*",
    ymin=1e-5},
    PlotInc({mark_size="1pt"},Table([1:m_mon,upper_mon])),
    PlotInc({mark_size="1pt"},Table([1:m_mon,err_mon])),    
)

In [ ]:
# DocumentPGFPlots.savefigs(joinpath(apath,"img","notmonotone"),Pnonmonotone)

In [ ]:
f = x->exp(cos(5x))
m = 30
err = zeros(m)
csize = zeros(m)
tol = 1e-14; cutoff = 1e-5
for N in 1:m
    P = ExtensionFramePlatform(platform(Fourier(1,-2,2)),-1.0..1.0)
    A = Fun(f, P, N;samplingstyle=GramStyle(),solverstyle=DirectStyle(),
        directsolver=regularized_SVD_solver, threshold=cutoff, discrete=false, rtol=tol, atol=tol)
    err[N] = L2error(f,A,  atol=tol, rtol=tol)
    csize[N] = norm(coefficients(A))
end

In [ ]:
upper = [NaN, (err[2:end].+sqrt(cutoff)*csize[2:end])...];

In [ ]:
P = @pgf Axis(
    {width=width,height=height,ymode="log",xlabel=L"$N$", 
    cycle_list_name="mark list*",
    ymin=1e-5},
    PlotInc({mark_size="1pt"},Table([1:m,upper])),
    PlotInc({mark_size="1pt"},Table([1:m,err])),    
)

In [ ]:
# DocumentPGFPlots.savefigs(joinpath(apath,"img","decreaseofL2norm"),P)

In [ ]:
f = x->1.
m_mon = 40
err_mon = zeros(m_mon)
csize_mon = zeros(m_mon)
tol = 1e-14; cutoff = 1e-5
for N in 1:m_mon
    P = ExtensionFramePlatform(platform(Fourier(1,-2,2)),-1.0..1.0)
    A = Fun(f, P, N;solverstyle=DirectStyle(),directsolver=regularized_SVD_solver, threshold=cutoff)
    err_mon[N] = L2error(f,A,  atol=tol, rtol=tol)
    csize_mon[N] = norm(coefficients(A))
end

In [ ]:
upper_mon =  [NaN, (err_mon[2:end].+(cutoff)*csize_mon[2:end])...];

In [ ]:
Pnonmonotone = @pgf Axis(
    {width=width,height=height,ymode="log",xlabel=L"$N$", 
    cycle_list_name="mark list*",
    ymax=1,ymin=1e-16},
    PlotInc({mark_size="1pt"},Table([1:m_mon,upper_mon])),
    PlotInc({mark_size="1pt"},Table([1:m_mon,err_mon])),    
)

In [ ]:
# DocumentPGFPlots.savefigs(joinpath(apath,"img","notmonotone2"),Pnonmonotone)

In [ ]:
f = x->exp(cos(5x))
m = 40
err = zeros(m)
csize = zeros(m)
tol = 1e-14; cutoff = 1e-5
for N in 1:m
    P = ExtensionFramePlatform(platform(Fourier(1,-2,2)),-1.0..1.0)
    A = Fun(f, P, N;solverstyle=DirectStyle(),directsolver=regularized_SVD_solver, threshold=cutoff)
    err[N] = L2error(f,A,  atol=tol, rtol=tol)
    csize[N] = norm(coefficients(A))
end

In [ ]:
upper = [NaN, (err[2:end].+(cutoff)*csize[2:end])...];

In [ ]:
P = @pgf Axis(
    {width=width,height=height,ymode="log",xlabel=L"$N$", 
    cycle_list_name="mark list*",
    ymin=1e-12},
    PlotInc({mark_size="1pt"},Table([1:m,upper])),
    PlotInc({mark_size="1pt"},Table([1:m,err])),    
)

In [ ]:
# DocumentPGFPlots.savefigs(joinpath(apath,"img","decreaseofL2norm2"),P)

# Adaptive vs optimal

## Adaptive vs optimal:  1-D FE

In [ ]:
using FrameFun, DomainSets, QuadGK, LaTeXStrings,PGFPlotsX, Statistics, QuadGK, ProgressBars
width = ".5\\textwidth"
height = ".3\\textwidth"
P = ExtensionFramePlatform(platform(Fourier(1,-2,2)),-1.0..1.0)
function getmedianindex(N)
    mindex = Array{Int}(undef,size(N)[1:end-1])
    for i in CartesianIndices(size(N)[1:end-1])
        mindex[i] = findfirst(median(N[i.I...,:]) .==N[i.I...,:])
    end
    mindex
end
function getmedian(N,mindex)
    NN = Array{eltype(N)}(undef,size(mindex))
    for i in CartesianIndices(size(NN))
        NN[i] = N[i.I...,mindex[i]]
    end
    NN
end

In [ ]:
ps = LinRange(10,500,20)
noexp = 7
Nmax = 2maximum(ps)

N1sfull = zeros(Int, length(ps), noexp)
totaltimefull = zeros(Float64, length(ps), noexp)
N2sfull = copy(N1sfull)
onetimefull = copy(totaltimefull)
ix = 1:length(ps)

for i in ProgressBar(ix)
    for n in 1:noexp
        p = ps[i]
        f = x->cos(p*x)
        _,t1,_ = @timed( F1 = Fun(f, P; numrandompts = 3, solverstyle=DirectStyle(),
                directsolver=regularized_SVD_solver, adaptivestyle = OptimalStyle(), 
                criterion = FNAStyle(),δ=1e-10,threshold=1e-12,maxlength = Nmax,maxiterations = 10000))
        totaltimefull[i,n] = t1
        N1sfull[i,n] = length(coefficients(F1))
        _,t2,_ = @timed(Fun(f, P,length(coefficients(F1)),solverstyle=DirectStyle(),
                directsolver=regularized_SVD_solver,threshold=1e-12))
        onetimefull[i,n] = t2
    end
end

In [ ]:
mindex = getmedianindex(N1sfull)
N1s = getmedian(N1sfull,mindex)
N2s = getmedian(N2sfull,mindex)
totaltime = getmedian(totaltimefull,mindex)
onetime = getmedian(onetimefull,mindex);

In [ ]:
P1 = @pgf Axis({xlabel=L"p",width=width,height=height,cycle_list_name="mark list*",},
    Plot(Table([ps[2:end],totaltime[:,1,1][2:end]./onetime[:,1,1][2:end]]))
    )

In [ ]:
for i in ProgressBar(ix)
    for n in 1:noexp
        p = ps[i]
        f = x->cos(p*x)
        F2 = Fun(f, P; adaptivestyle = GreedyStyle(), solverstyle=DirectStyle(),
            directsolver=regularized_SVD_solver, numrandompts = 3,p0=max(N1s[i]-10,4), 
            criterion = FNAStyle(),maxlength = Nmax,maxiterations = 10000)
        N2sfull[i,n] = length(coefficients(F2))
        @assert N2sfull[i,n] > max(N1s[i]-10,4)
    end
end
N2s = getmedian(N2sfull,mindex);

In [ ]:
P2 = @pgf Axis({cycle_list_name="mark list*",xlabel=L"p",width=width,height=height,ylabel=L"\Delta N"},
    PlotInc(Table([ps,N1s.-N2s])),
)

In [ ]:
# DocumentPGFPlots.savefigs(joinpath(apath,"img","adaptiveVsOptimal"),P1)
# DocumentPGFPlots.savefigs(joinpath(apath,"img","adaptiveOptimalN"),P2)

## Adaptive vs optimal:  2-D Weighted FE

In [ ]:
using FrameFun, DomainSets, LaTeXStrings,PGFPlotsX, Statistics
p = IncrementalCartesianParameterPath{2}()
P = ExtensionFramePlatform(WeightedSumPlatform(platform(ChebyshevT(10,-1,1)^2), (x,y)->1.,
            (x,y)->sqrt(x^2+y^2)),.9*UnitDisk())
path = HierarchyPath(p,ProductPath(p,p))
paramP = parametrizedplatform(P, path)
function getmedianindex(N)
    mindex = Array{Int}(undef,size(N)[1:end-1])
    for i in CartesianIndices(size(N)[1:end-1])
        mindex[i] = findfirst(median(N[i.I...,:]) .==N[i.I...,:])
    end
    mindex
end
function getmedian(N,mindex)
    NN = Array{eltype(N)}(undef,size(mindex))
    for i in CartesianIndices(size(NN))
        NN[i] = N[i.I...,mindex[i]]
    end
    NN
end

In [ ]:
ps = LinRange(0,3,10)
noexp = 1
Nmax = 3000
N1sfull = zeros(Int, length(ps), noexp)
totaltimefull = zeros(Float64, length(ps), noexp)
onetimefull = copy(totaltimefull)

for (i,p) in enumerate(ps)
    for n in 1:noexp
        @show p, noexp
        f = (x,y) -> cos(p*pi*(x+y)) + sqrt(x^2+y^2)*sin(1+p*pi*(x+y))
        _,t1,_ = @timed( F1 = Fun(f, paramP; numrandompts = 3,adaptivestyle = OptimalStyle(), 
                criterion = FNAStyle(),δ=1e-6, threshold=1e-8, maxlength = Nmax,maxiterations = 10000))
        totaltimefull[i,n] = t1
        N1sfull[i,n] = length(coefficients(F1))
        _,t2,_ = @timed(F=Fun(f, P,tuple(dimensions(dictionary(F1))...), threshold=1e-8, ))
        onetimefull[i,n] = t2
    end
end

In [ ]:
mindex = getmedianindex(N1sfull)
N1s = getmedian(N1sfull,mindex)
totaltime = getmedian(totaltimefull,mindex)
onetime = getmedian(onetimefull,mindex);

In [ ]:
P1 = @pgf GroupPlot({group_style = {group_size="2 by 1",},cycle_list_name="mark list*"},
    {width=width,height=height,xlabel=L"p",ylabel=L"N",ymin=0},
    Plot(Table([ps,N1s])),
    {legend_cell_align="left",width=width,height=height,ymin=0,ymax=20,xlabel=L"p"},
    Plot(Table([ps[2:end],totaltime[2:end]./onetime[2:end]])),
)

In [ ]:
# DocumentPGFPlots.savefigs(joinpath(apath,"img","adaptiveVsOptimal2dwFE"),P1)

In [ ]:
f = (x,y) -> cos(last(ps)*pi*(x+y)) + sqrt(x^2+y^2)*sin(1+last(ps)*pi*(x+y))
F = Fun(f, paramP; numrandompts = 3,adaptivestyle = OptimalStyle(), 
    criterion = FNAStyle(),δ=1e-6, threshold=1e-8,maxlength = Nmax,maxiterations = 10000,verbose=false)

In [ ]:
using Plots
x = EquispacedGrid(100,-1,1)
Plots.plot(F;c=:RdBu,size=(2*300,2*140),layout=2)
wcP = Plots.heatmap!(log10.(eps().+abs.(F(x^2)-[f(xi, yi) for xi in x, yi in x]));
    subplot=2,aspect_ratio=1,ticks=false)

In [ ]:
# savefig(wcP, joinpath(apath,"img","weightedcircle"))

# Coefficient norm

In [ ]:
using BasisFunctions, FrameFun, DomainSets, PGFPlotsX, Plots, LaTeXStrings, DocumentPGFPlots

In [ ]:
width = ".5\\textwidth"
height = ".3\\textwidth"

In [ ]:
ymax = 1e6; ymin = 1e-15
epss = exp10.(LinRange(-12,-3,4))
T = 2; sampling_factor=2; 
t = 1;
@show epss

# exp(cos(8pix))

In [ ]:
f_1 = x->exp(cos(8pi*x))
# f_2 = x->1/(x-1.01)

In [ ]:
ns_1 = 2collect(1:10:200) .+ 1
csize_1 = zeros(Float64,length(ns_1),length(epss))
L2err_1 = zeros(Float64,length(ns_1),length(epss))
l2err_1 = zeros(Float64,length(ns_1),length(epss))
fsize_1 = zeros(Float64,length(ns_1),length(epss))
minN_1 = zeros(Int,length(epss))
P = ExtensionFramePlatform(platform(Fourier(1, -T,T)),Interval(-t,t))
for (i,n) in enumerate(ns_1) 
    for (j,epsilon) in enumerate(epss)        
        F, A, b, c, S, _  = approximate(f_1,P,n;solverstyle=DirectStyle(),
            directsolver=pQR_solver, L=sampling_factor*n,threshold=epsilon)
        csize_1[i,j] = norm(c)
        L2err_1[i,j] = L2error(f_1, F; atol=1e-5,rtol=1e-5)
        b = S*f_1
        l2err_1[i,j] = norm(b-A*c)
        fsize_1[i,j] = sqrt(step(supergrid(supergrid(FrameFun.grid(element(S,1))))).*norm(b)^2)
        (minN_1[j] == 0) && (l2err_1[i,j] < epsilon*fsize_1[i,j]) && (minN_1[j] = n)
    end
end

In [ ]:
@pgf p_1 = GroupPlot(
    {width=width,height=height,ymode="log", 
    cycle_list_name="mark list*",
    xlabel=L"$N$", legend_pos="south west",legend_cell_align="left",
        ymin=1e-13,ymax=1e12,group_style={group_size={"2 by 2"},
        x_descriptions_at="edge bottom",
        y_descriptions_at="edge left", 
        horizontal_sep="1em",vertical_sep="1em",
        
    }},
    )
@pgf for (j,epsilon) in enumerate(epss)
    push!(p_1,{})
    push!(p_1,PlotInc({},Table(ns_1,L2err_1[:,j])))
    push!(p_1,PlotInc({},Table(ns_1,l2err_1[:,j])))
    push!(p_1,PlotInc({},Table(ns_1,csize_1[:,j])))
    push!(p_1,PlotInc({style="black",mark="none"},Table(ns_1,fsize_1[:,j]))) 
    push!(p_1,PlotInc({style="black,dashed",mark="none"},Table(ns_1, epsilon*ones(size(ns_1)))))
end
p_1

In [ ]:
# DocumentPGFPlots.savefigs(joinpath(apath,"img","groupplotexpcos8pix"),p_1)

# Noise

Noise is not seen

In [ ]:
P = ExtensionFramePlatform(FourierPlatform(),0.0..0.5)

In [ ]:
f = x->exp(x) + 1e-3*cos(2pi*1000x)
F = Fun(f,P;threshold=1e-10/100,δ=1e-10,solverstyle=DirectStyle(),directsolver=pQR_solver,
    criterion=FNAStyle(),verbose=false)

In [ ]:
δs = 10. .^ LinRange(-10,-3,20)
ϵs = 10. .^ LinRange(-10,-3,20)
ls = zeros(length(δs),length(ϵs))
ns = zeros(length(δs),length(ϵs))
es = zeros(length(δs),length(ϵs))
# for (i,δ) in enumerate(δs)
#     for (j,ϵ) in enumerate(ϵs)
#         f = x->exp(x) + ϵ*cos(2pi*1000x)
#         F = Fun(f,P;threshold=δ/100,δ=δ,criterion=FNAStyle(),solverstyle=DirectStyle(),
#             directsolver=pQR_solver,verbose=false)
#         N = length(coefficients(F))
#         _,A,b,c,S,L = approximate(f,P,N;threshold=δ/100,δ=δ,solverstyle=DirectStyle(),
#             directsolver=pQR_solver)
#         ls[i,j] = length(F)
#         ns[i,j] = norm(coefficients(F))
#         es[i,j] = norm(A*c-b)
#         @show δ, ϵ, length(F), norm(coefficients(F)), norm(A*c-b)
#     end
# end   

ls = [1989.0 1991.0 1993.0 1993.0 1995.0 1995.0 1997.0 1997.0 1999.0 1999.0 1999.0 2001.0 2001.0 2001.0 2001.0 2001.0 2001.0 2001.0 2001.0 2001.0; 49.0 223.0 1991.0 1993.0 1995.0 1995.0 1997.0 1997.0 1997.0 1999.0 1999.0 1999.0 2001.0 2001.0 2001.0 2001.0 2001.0 2001.0 2001.0 2001.0; 37.0 49.0 119.0 49.0 1993.0 1995.0 1995.0 1997.0 1997.0 1997.0 1999.0 1999.0 1999.0 2001.0 2001.0 2001.0 2001.0 2001.0 2001.0 2001.0; 23.0 37.0 49.0 1991.0 1993.0 1995.0 1995.0 1995.0 1997.0 1997.0 1999.0 1999.0 1999.0 2001.0 2001.0 2001.0 2001.0 2001.0 2001.0 2001.0; 23.0 23.0 37.0 49.0 1991.0 1993.0 49.0 1995.0 1997.0 1997.0 1997.0 1999.0 1999.0 2001.0 2001.0 2001.0 2001.0 2001.0 2001.0 2001.0; 21.0 21.0 21.0 37.0 49.0 1993.0 1993.0 49.0 1995.0 1997.0 1997.0 1997.0 1999.0 1999.0 2001.0 2001.0 2001.0 2001.0 2001.0 2001.0; 21.0 21.0 21.0 21.0 37.0 49.0 223.0 1993.0 1995.0 1995.0 1997.0 1997.0 1999.0 1999.0 2001.0 2001.0 2001.0 2001.0 2001.0 2001.0; 21.0 21.0 21.0 21.0 21.0 37.0 49.0 1993.0 1995.0 1995.0 1997.0 1997.0 1999.0 1999.0 1999.0 1999.0 2001.0 2001.0 2001.0 2001.0; 18.0 18.0 18.0 18.0 18.0 18.0 37.0 49.0 1993.0 1995.0 1997.0 1997.0 1997.0 1999.0 1999.0 2001.0 2001.0 2001.0 2001.0 2001.0; 18.0 18.0 18.0 18.0 18.0 18.0 18.0 37.0 49.0 223.0 1995.0 1997.0 1997.0 1999.0 1999.0 1999.0 1999.0 2001.0 2001.0 2001.0; 17.0 17.0 17.0 17.0 17.0 17.0 17.0 17.0 17.0 17.0 49.0 1997.0 1997.0 1997.0 1999.0 1999.0 1999.0 2001.0 2001.0 2001.0; 17.0 17.0 17.0 17.0 17.0 17.0 17.0 17.0 17.0 17.0 17.0 17.0 1997.0 1999.0 1999.0 1999.0 1999.0 1999.0 2001.0 2001.0; 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 1999.0 2001.0 2001.0 2001.0 2001.0 2001.0 2001.0; 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 15.0 119.0 2001.0 2003.0 2001.0 2003.0 2003.0 2001.0; 13.0 13.0 13.0 13.0 13.0 13.0 13.0 13.0 13.0 13.0 13.0 13.0 13.0 15.0 49.0 1999.0 1999.0 2001.0 2005.0 2003.0; 13.0 13.0 13.0 13.0 13.0 13.0 13.0 13.0 13.0 13.0 13.0 13.0 13.0 13.0 15.0 2007.0 119.0 2009.0 2001.0 2001.0; 10.0 10.0 10.0 10.0 10.0 10.0 10.0 10.0 10.0 10.0 10.0 10.0 10.0 10.0 10.0 15.0 2011.0 2001.0 2011.0 2001.0; 10.0 10.0 10.0 10.0 10.0 10.0 10.0 10.0 10.0 10.0 10.0 10.0 10.0 10.0 10.0 10.0 15.0 49.0 2011.0 2001.0; 9.0 9.0 9.0 9.0 9.0 9.0 9.0 9.0 9.0 9.0 9.0 9.0 9.0 9.0 9.0 9.0 10.0 15.0 49.0 49.0; 9.0 9.0 9.0 9.0 9.0 9.0 9.0 9.0 9.0 9.0 9.0 9.0 9.0 9.0 9.0 9.0 9.0 9.0 15.0 2001.0]
ns = [1.0921747809707683 0.9640240487170346 0.9531240774091445 1.0141706818805694 0.9816293347274048 1.242075430465959 0.9452139652698703 0.9888483765530071 0.9334464613854546 0.9413036322000337 0.9866365864162162 0.9321646666556709 0.9321747242972328 0.932209005347928 0.9321953094346858 0.9322052541773826 0.932157106241263 0.9322469019840033 0.9321509350562402 0.9322015741295727; 1.1551809041151622 0.9721237181348876 1.045153195396157 1.0350793023385467 0.9381671888581531 0.9929833983108666 0.9412826533167956 0.9419822231120075 1.2030278724047445 0.9353962531111092 0.9441974201666075 0.9952523847838488 0.9318620498559878 0.9318913789609017 0.9321779535856581 0.9321608762296424 0.9321628212805599 0.9321188195902195 0.9321685307178866 0.9321479201959392; 1.2886389313341788 1.1218490573982958 1.0083187999829561 1.122694038227333 1.0252865304053433 0.941321634493643 1.1426393226221199 0.9373396605252298 0.9749679599158392 1.156348720282568 0.9485024082730241 0.9437622102642095 1.2649160580858076 0.9318538100416434 0.9318678783403785 0.9318603499148055 0.9318710956922744 0.9318893707194882 0.9318967666358572 0.9318743672415765; 1.371490295312085 1.3642654315741343 1.1220018895975943 1.0034245952586516 0.9433151785893531 0.9407996982070563 0.9842407790118695 1.173277919504205 0.9442926608543828 1.1463867050365288 0.9333528013689935 0.9418269802424146 1.020872875810054 0.9318648986844066 0.9315487515473219 0.9317919064125717 0.9315962442473461 0.9315645409540109 0.9318249398044783 0.9318737785837143; 1.3714903009277954 1.3714902969098275 1.178226150201492 1.1217741458947796 1.2143763917913817 1.0090193023205265 1.1187492162321329 0.9676671437886536 0.9406498098356929 0.9914705178897999 1.1344081597165436 0.9357524328173787 0.9985158690733434 0.9315355994663783 0.9315534887585024 0.9315699889836386 0.9315558640507117 0.9316124517517562 0.9315582001295403 0.9315773024868831; 1.3708656147685665 1.371289075537937 1.3722799962116976 1.174713705763305 1.1157766919806966 0.9443941386277602 1.0009181664175406 1.120529396571762 1.1114608501965142 0.9396382348934718 1.1676146712222342 1.1605920393240765 0.9521415440084103 1.0119480784139572 0.9312727191647361 0.931549047012892 0.931560412576018 0.931243868841889 0.9315476953111324 0.9312557249461887; 1.3708656145895917 1.3712890763733065 1.3722799945864357 1.3746044825055554 1.1528879585759875 1.1143478564060834 0.9638545378957598 1.1498649880745246 0.9598646128492094 1.5068385656241434 0.9701567656756072 1.155310302359801 0.9335147261849546 0.9461564432658448 0.9312069657131481 0.9312594369106266 0.9312356435499837 0.9312184528352035 0.9312468589655545 0.9312423929076388; 1.370865614478127 1.3712890748828919 1.3722799955642313 1.3746044832205844 1.3800879117661478 1.1355060019877923 1.0885650980251873 0.9885052101527537 0.9592910520325801 1.0652347118439442 0.94121429018266 1.1272486284391434 0.9331738620838186 0.942249911697412 1.0117534805239363 1.332092642237815 0.9312433778985176 0.9309439787835575 0.9309575711161835 0.9309340387166556; 1.3691495007779015 1.3691473161240046 1.3691422133920663 1.3691302950292479 1.369102458462174 1.3690374470313387 1.1538070929957833 1.088772243094623 0.9587371518872057 0.947435023564159 0.9416121663757947 0.9865658758893435 1.1309278698078402 0.9359942277395772 0.9539276662122262 0.9308878420572614 0.9309223778051429 0.9309361383152203 0.9306237721443721 0.9309385415910556; 1.369149500803796 1.369147316115568 1.369142213389971 1.3691302950637498 1.3691024584560219 1.3690374470126399 1.3688856374155862 1.1414782579123925 1.0947229875394584 0.9595192188306803 1.0498706236450999 0.9726814276857136 1.1171767155397125 0.9366768391926019 0.9516233922088413 1.02191177520511 1.3520596205015953 0.9306845786502703 0.9306497921337753 0.93064413187114; 1.3681979300081233 1.368196559294833 1.368193357711566 1.3681858797897486 1.3681684139759551 1.3681276216142635 1.3680323577240734 1.3678099333499616 1.3672908794470344 1.3660810675314607 1.0838781302065765 0.9359833102942964 0.9701834547801311 1.1097449528001262 0.9333370263749916 0.9642589628704904 1.0497351236228756 0.9306610085734376 0.930583484102656 0.9306494964280728; 1.3681979300167129 1.368196559270086 1.3681933577551584 1.3681858797577624 1.3681684140089012 1.368127621618525 1.3680323577321578 1.3678099333806257 1.367290879408545 1.3660810675984225 1.363269261374381 1.3567785134995995 0.9379266029671207 0.9306852822142758 0.9331803670062802 0.9475731551942828 1.0529694896753308 1.3831010457727377 0.930334315561973 0.9303537288613185; 1.3607418260971897 1.3607418261034274 1.3607418261073685 1.3607418261175541 1.3607418261536786 1.360741826237011 1.3607418264387217 1.3607418269140767 1.360741828004322 1.360741830549134 1.3607418365063904 1.3607418504152877 1.360741882898622 0.9298996612396163 0.9303055509974902 0.9303274955899099 0.9303508385513941 0.9303525680082079 0.9300361650284463 0.9303334275192413; 1.3607418261055628 1.3607418260939281 1.360741826099791 1.3607418261187854 1.3607418261506476 1.36074182623511 1.3607418264443798 1.3607418269152536 1.3607418279992658 1.3607418305490548 1.3607418365072017 1.3607418504136914 1.3607418829041789 0.9801653068293397 0.9295127593683384 0.9301950405830851 0.9297933813690168 0.9299128372620009 0.9296461535077738 0.9297133766643241; 1.3579711111498245 1.3579713103638318 1.3579717756738245 1.357972862505111 1.3579754010544445 1.3579813304465778 1.3579951801199117 1.3580275305541056 1.358103100083676 1.3582796526795187 1.3586922660881307 1.359657300592968 1.3619183414579759 1.3607419587869034 1.058841597253233 0.9301227182319208 0.931381733983712 0.9295551243425684 0.929741068802944 0.9295609772804209; 1.357971111149515 1.3579713103628939 1.3579717756728726 1.3579728625057028 1.3579754010543086 1.3579813304471842 1.3579951801207573 1.3580275305551437 1.3581031000831436 1.3582796526785867 1.3586922660872882 1.3596573005934385 1.3619183414580074 1.367237394767336 1.360742136063133 0.9295918908630604 0.9732493765994663 0.9296882128678212 0.9295639655121289 0.9294675268785211; 1.3616062035981542 1.3616062101332742 1.361606225397469 1.3616062610503878 1.3616063443256614 1.361606538833624 1.3616069931501942 1.3616080543084035 1.361610532885155 1.3616163221863833 1.361629844579716 1.361661430205587 1.361735211329365 1.3619075756618366 1.3623103461983668 1.3607425502211734 0.9295786449391904 0.929261359606099 0.929548338152947 0.9292405134443579; 1.3616062035981646 1.3616062101333 1.3616062253974657 1.3616062610503687 1.3616063443256898 1.361606538833586 1.3616069931501862 1.3616080543084352 1.361610532885151 1.36161632218639 1.3616298445797315 1.3616614302055834 1.3617352113293675 1.3619075756618524 1.3623103461983852 1.3632520621871707 1.3607435180907692 1.05871345567033 0.9296015472004129 0.9291931679245617; 1.3595855332360745 1.3595855398456786 1.359585555283931 1.3595855913434032 1.3595856755682545 1.359585872294094 1.3595863317911963 1.3595874050496508 1.3595899118892634 1.3595957672055003 1.3596094437962467 1.3596413896097481 1.3597160121698388 1.3598903426181004 1.3602977095687754 1.361250183912979 1.3654568515260535 1.360745781660688 1.0232575037415768 1.0277198841478488; 1.3595855332360425 1.3595855398456738 1.359585555283924 1.3595855913433794 1.3595856755682614 1.3595858722941154 1.3595863317911911 1.359587405049632 1.359589911889252 1.35959576720549 1.3596094437962598 1.3596413896097679 1.3597160121698448 1.359890342618127 1.3602977095688125 1.361250183912944 1.3634802235488843 1.368717846857617 1.3607510844105724 0.928895287239631]
es = [8.922542309243618e-11 4.7334661223803675e-11 3.521095876977903e-11 7.524501116257138e-11 2.1062357727435843e-11 5.774886388256852e-11 1.7357894022002382e-11 3.9767045309193724e-11 5.2655613477882305e-12 8.753481501783594e-12 2.112576955763118e-11 2.6250549737551498e-12 9.549793695831674e-11 2.2183607744839563e-12 2.0608695899412907e-12 2.1247161502246293e-12 2.0589836661120726e-12 2.4496733551127104e-12 2.3044320459767826e-12 1.888636788978767e-12; 4.339238197721112e-13 4.736933829781389e-12 1.8715327763289832e-10 1.357283038289308e-10 4.271050851734098e-11 1.0544303257444942e-10 2.2204049027109865e-11 5.1679870375240144e-11 2.1845806535426538e-10 1.617204185022955e-11 4.8506085369867094e-11 1.0364969196273175e-10 1.3864762986884768e-11 3.0074271589363527e-12 8.031078461212933e-12 3.403111223382911e-12 1.7627883063777077e-11 2.5114056307810974e-12 3.1182919701344335e-12 2.7304426532206626e-12; 3.321360304494854e-12 1.7376398074067695e-11 3.2096926302375705e-11 2.4368563789500736e-11 3.301974338078751e-10 2.726026566259126e-10 5.265693771075527e-10 9.712396961591138e-11 2.1584270598942845e-10 4.6525833045732155e-10 4.687783133534187e-11 2.414266254385897e-10 6.50542060728202e-10 3.855236168704642e-11 1.1298167275973553e-11 8.85099393927128e-12 9.947910508838925e-12 8.342554968416246e-12 1.2146827962160124e-11 7.828117970261115e-12; 9.065851224365392e-10 1.0019749335805675e-10 1.6438705831248886e-11 8.804749814326085e-10 6.855974761025631e-10 1.8577994409953733e-10 4.707029454821184e-10 1.0841971762211115e-9 3.9567962483150673e-10 5.055891343827266e-10 1.4205472146700097e-10 2.434545936425292e-10 6.251557079120524e-10 1.8508714471099593e-11 1.0477751051912438e-11 1.0193229414614705e-11 2.292027600042226e-11 1.1588641010521105e-10 5.21468336158994e-11 2.009216992956287e-10; 9.065849173734935e-10 9.065847734089523e-10 1.0805376099382441e-10 3.37941244063831e-11 3.3710240743708016e-9 1.5612334602818292e-9 3.6499070025637005e-11 2.142255108739171e-9 4.120447001217274e-10 1.0428283073364982e-9 2.7603975371966554e-9 4.847023376749098e-10 1.2988935601203198e-9 5.15582644934376e-11 2.942887376858672e-10 6.101157151291283e-11 5.275444591721445e-11 4.892788203457784e-11 5.7810529724926135e-11 1.3248109873339383e-10; 5.718770996398609e-9 5.736667137380233e-9 5.837030695111108e-9 1.9185798307511723e-10 4.1278757841459396e-11 1.4438091834662092e-9 3.3573151688262124e-9 3.31702956851328e-11 4.992140835060758e-9 9.909302610226652e-10 5.04081137304057e-9 1.1716272723639882e-8 1.224239357641775e-9 2.7439907632878773e-9 3.411859789495704e-10 1.0561900150709336e-9 5.6643808473299194e-11 6.134932032370486e-10 2.046872684005689e-10 7.887835245034907e-10; 5.7187712803097535e-9 5.736667257746214e-9 5.837030825457056e-9 6.3646539497491525e-9 2.193482579971119e-9 3.5880920387097964e-11 1.0899830229459994e-9 1.433355297213204e-8 9.145937941555256e-9 1.137488899260395e-8 5.417274075741507e-9 1.3227913605654865e-8 3.2694634715895052e-9 7.284374918627624e-9 2.4630715357970214e-9 2.6274538292235475e-10 1.960916142961745e-10 3.4232071342456643e-10 8.791047223153302e-10 2.544719291329915e-10; 5.718771162523173e-9 5.736666959529274e-9 5.837030802932511e-9 6.364653961609837e-9 8.710151641776607e-9 8.91738122863316e-10 1.0571664505717676e-9 2.9362970727163776e-8 1.1713498918322768e-8 2.34802185847966e-8 1.2938315100353159e-8 2.277296071332389e-8 3.1504436581495352e-9 6.9666355554497265e-9 1.551345305622249e-8 3.5347683391496075e-8 4.450412777194885e-9 1.2763172834267186e-9 1.1121901801859658e-9 1.074631316757669e-9; 3.259606516661983e-8 3.2597656209224564e-8 3.263386518856332e-8 3.289453146226196e-8 3.4424342574048424e-8 4.2068593274314635e-8 2.0079378894052575e-9 1.0814983440894297e-9 6.442631142300108e-8 4.2337659182121545e-8 1.1353006392400716e-8 2.4842303473776165e-8 5.666994529050362e-8 1.2884982325622327e-8 3.096184403797681e-8 1.2995360962077547e-9 1.483393630437775e-9 6.549974827674613e-9 2.472331964741084e-9 4.2272279548307665e-9; 3.259606528652789e-8 3.259765611112994e-8 3.263386566896962e-8 3.289453106035492e-8 3.442434280617078e-8 4.2068593013602665e-8 7.080450265923709e-8 1.2236150858148395e-8 1.0887929163939703e-8 5.7518128285804984e-8 1.041966017379924e-7 4.889549199408922e-8 1.1377863965300315e-7 1.929427855730364e-8 3.934251643476528e-8 8.02769319628333e-8 4.41484415569719e-7 3.9887416089191565e-9 5.856544840565696e-9 5.946874952338252e-9; 2.2695487750088114e-7 2.269549221939083e-7 2.2695502779066068e-7 2.269552804064527e-7 2.2695590432702204e-7 2.2695754707439984e-7 2.269623920051002e-7 2.2697920981442258e-7 2.27048493874754e-7 2.273737575774555e-7 2.28338404014909e-9 5.275319673738466e-8 1.224360956104433e-7 2.73025504677738e-7 7.781306605755374e-8 1.615861166929472e-7 1.708813576981668e-7 2.4615166818049577e-8 6.385767334253777e-9 2.2813341498231837e-8; 2.269548774637097e-7 2.2695492226488498e-7 2.2695502766799334e-7 2.2695528016143415e-7 2.2695590436036102e-7 2.269575469935264e-7 2.2696239217887283e-7 2.269792101637249e-7 2.2704849367113602e-7 2.2737375755644326e-7 2.290187587973096e-7 2.375196424216249e-7 2.069259347958511e-7 4.493245585226861e-8 7.788279364242004e-8 1.836650133439634e-7 7.681830209201027e-7 1.847225243013257e-6 2.378837657895875e-8 4.924697383595751e-8; 1.0670748647366955e-6 1.0670748646993297e-6 1.06707486457533e-6 1.0670748646700187e-6 1.0670748647503767e-6 1.0670748653623432e-6 1.067074864759884e-6 1.0670748647322496e-6 1.0670748646653287e-6 1.0670748644004806e-6 1.0670748645736706e-6 1.0670748643318902e-6 1.0670748649015512e-6 1.796464913723443e-6 9.051198666826255e-8 3.609454742676246e-8 3.612795367451919e-8 5.4771395269666165e-8 4.2703116127461186e-8 7.471404283987068e-8; 1.0670748645813096e-6 1.0670748651427163e-6 1.0670748641127323e-6 1.06707486462239e-6 1.0670748646606978e-6 1.06707486498383e-6 1.067074864893881e-6 1.0670748647486077e-6 1.067074864778017e-6 1.0670748650323049e-6 1.0670748649164937e-6 1.0670748647673271e-6 1.067074864624751e-6 2.008507306492301e-7 8.823274403519177e-8 2.2943980406119464e-7 3.09460531327348e-5 2.7159888117087374e-7 6.162195966511452e-8 8.28590016996648e-6; 6.703090196729213e-6 6.703106881235047e-6 6.703145966652174e-6 6.703237885553262e-6 6.703456001434307e-6 6.703984105963426e-6 6.705319311877668e-6 6.708992383342792e-6 6.720589395988135e-6 6.764015657045077e-6 6.95196366900314e-6 7.807239437597836e-6 1.1245213327933021e-5 1.0670748648304474e-6 1.1012931592823472e-7 1.0570658345716749e-5 1.4574939754078915e-5 5.842075059267454e-5 1.8869725371763046e-7 0.00013570526516998592; 6.703090196804475e-6 6.70310688142986e-6 6.7031459665286836e-6 6.703237885735718e-6 6.703456001433993e-6 6.7039841059675445e-6 6.70531931192938e-6 6.708992383213634e-6 6.72058939582039e-6 6.764015656912178e-6 6.9519636691383416e-6 7.807239437739685e-6 1.124521332780776e-5 2.181619119621083e-5 1.0670748650425138e-6 2.2596211080525303e-6 3.2137895848263136e-6 6.718161024185933e-7 6.963915496785401e-5 2.4919395859325872e-5; 5.079922235670179e-5 5.079922842695402e-5 5.079924261579061e-5 5.0799275816227726e-5 5.0799353682188706e-5 5.079953729923131e-5 5.0799975691459856e-5 5.080105155406255e-5 5.080384759499311e-5 5.081192264354268e-5 5.0839202968434234e-5 5.0948742638964586e-5 5.145201186047031e-5 5.3914347053026175e-5 6.535666173881587e-5 1.0670748645262827e-6 2.856157994066629e-6 7.093044977736685e-5 1.2527763000408432e-6 7.670441847770723e-5; 5.079922235663661e-5 5.0799228426753206e-5 5.079924261611842e-5 5.079927581601672e-5 5.079935368210346e-5 5.079953729947521e-5 5.079997569138071e-5 5.080105155415751e-5 5.0803847595152384e-5 5.081192264336578e-5 5.0839202968498134e-5 5.094874263928937e-5 5.145201186048236e-5 5.3914347052995214e-5 6.535666173902491e-5 0.00010769639864396523 1.0670748647229416e-6 7.324161594212414e-6 5.223401783639416e-6 0.0003117144361895008; 0.00035473206054557156 0.00035473206202494 0.0003547320654820817 0.00035473207356472616 0.00035473209248489693 0.00035473213690312953 0.00035473224188585235 0.0003547324938275031 0.0003547331190177087 0.0003547347796370202 0.00035473975139095154 0.00035475732667560717 0.0003548309000219502 0.00035517999660243676 0.0003569576252196557 0.0003662286874760629 0.00022648434309803652 1.0670748645619748e-6 2.766138596757125e-5 2.688432692399812e-5; 0.00035473206054529514 0.00035473206202470836 0.0003547320654821608 0.0003547320735645913 0.0003547320924848335 0.00035473213690321415 0.00035473224188589637 0.0003547324938275366 0.00035473311901748964 0.00035473477963726213 0.0003547397513909363 0.00035475732667547723 0.0003548309000217415 0.00035517999660258546 0.0003569576252195872 0.00036622868747631824 0.00041259020519786023 0.0006050761195338793 1.0670748645482914e-6 0.0006466900002674653];

In [ ]:
using Pkg;Pkg.activate(localprojectdir());using PGFPlotsX, LaTeXStrings, DocumentPGFPlots
opts = @pgf {
        view = (0, 90),
        xlabel=L"\log_{10}(\delta)",
        ylabel=L"\log_{10}(\sigma)",
        colorbar,
        "colormap/jet",
        width = ".25\\textwidth",
        scale_only_axis=true,
    }
p3opts = @pgf {surf,shader = "flat"}

In [ ]:
P = @pgf GroupPlot({opts...,group_style={
        x_descriptions_at="edge bottom",
        y_descriptions_at="edge left",
        horizontal_sep="5em",group_size="3 by 1",}},
    {},Plot3(p3opts,Table(log10.(ϵs), log10.(δs),log10.(es))),
    {},Plot3(p3opts,Table(log10.(ϵs), log10.(δs),log10.(ls))),
    {},Plot3(p3opts,Table(log10.(ϵs), log10.(δs),log10.(ns))),
)

In [ ]:
# DocumentPGFPlots.savefigs(joinpath(apath,"img","svsd"), P)

## \epsilon vs \delta

In [ ]:
f = x-> 1e6exp(cos(8pi*x))
P = ExtensionFramePlatform(FourierPlatform(),0.0..0.5)

In [ ]:
δs = 10. .^ LinRange(-10,-3,20)
ϵs = 10. .^ LinRange(-10,-3,20)
ls = zeros(length(δs),length(ϵs))
ns = zeros(length(δs),length(ϵs))
# This computation takes a very long time. Consider it running on a computation server. (Or use the AZ version)
# for (i,δ) in enumerate(δs)
#     for (j,ϵ) in enumerate(ϵs)
#         F = Fun(f,P;threshold=ϵ,δ=δ,solverstyle=DirectStyle(),directsolver=pQR_solver,criterion=FNAStyle(),
#             verbose=false)
#         @show δ, ϵ, length(F), norm(coefficients(F))
#         ls[i,j] = length(F)
#         ns[i,j] = norm(coefficients(F))
#     end
# end

ls = [257.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 103.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 71.0 91.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 449.0 79.0 989.0 257.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 71.0 67.0 95.0 505.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 63.0 63.0 65.0 65.0 65.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 61.0 61.0 61.0 61.0 217.0 251.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 59.0 61.0 61.0 61.0 61.0 61.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 57.0 57.0 57.0 59.0 59.0 59.0 61.0 4049.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 55.0 55.0 55.0 55.0 57.0 57.0 57.0 99.0 101.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 53.0 53.0 53.0 53.0 53.0 53.0 55.0 55.0 57.0 61.0 65.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 49.0 51.0 51.0 51.0 51.0 51.0 51.0 51.0 53.0 651.0 481.0 63.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 47.0 47.0 47.0 47.0 49.0 49.0 51.0 51.0 51.0 51.0 51.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 45.0 45.0 45.0 45.0 45.0 47.0 47.0 47.0 49.0 49.0 49.0 51.0 1001.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 43.0 43.0 43.0 43.0 43.0 43.0 43.0 43.0 45.0 45.0 47.0 47.0 47.0 49.0 59.0 4097.0 4097.0 4097.0 4097.0 4097.0; 39.0 39.0 41.0 41.0 41.0 43.0 41.0 43.0 43.0 43.0 43.0 45.0 45.0 45.0 127.0 4097.0 4097.0 4097.0 4097.0 4097.0; 35.0 37.0 37.0 37.0 39.0 39.0 41.0 41.0 41.0 41.0 41.0 41.0 41.0 43.0 233.0 507.0 4097.0 4097.0 4097.0 4097.0; 33.0 34.0 35.0 35.0 35.0 37.0 37.0 37.0 39.0 39.0 39.0 39.0 39.0 39.0 71.0 41.0 113.0 41.0 4097.0 4097.0; 31.0 31.0 31.0 31.0 33.0 33.0 33.0 33.0 35.0 35.0 35.0 37.0 37.0 37.0 37.0 37.0 109.0 37.0 4097.0 4097.0; 29.0 29.0 29.0 29.0 29.0 29.0 31.0 31.0 31.0 33.0 33.0 33.0 35.0 35.0 35.0 35.0 35.0 39.0 4097.0 4097.0]
ns = [1.1711670998275564e6 1.074741210683931e6 1.0747155734131627e6 1.0743114538204882e6 1.07393591498088e6 1.0738266183818155e6 1.0734761666306527e6 1.0733681654535052e6 1.0729878039496355e6 1.0725209485917217e6 1.0725362807475247e6 1.072179213666102e6 1.071658661963075e6 1.071651846848696e6 1.0712830436383812e6 1.0713039395513583e6 1.0708424409113151e6 1.0704318749921443e6 1.070079591894056e6 1.0700877870287835e6; 1.2689594071905438e6 1.074829570190958e6 1.0746706624008177e6 1.0743518118510763e6 1.0738410690772785e6 1.0738646148922415e6 1.0735362913972687e6 1.0733257092370237e6 1.0729522477355092e6 1.0725626464204094e6 1.0726001321468817e6 1.0720621075762243e6 1.0721476956476036e6 1.0716719410392959e6 1.071336171034196e6 1.071140514446898e6 1.0707527868262138e6 1.070528664052692e6 1.0701264996754087e6 1.0700460567820484e6; 1.2979272450885219e6 1.2711659595567556e6 1.074751879467742e6 1.07435034957139e6 1.0738871588654374e6 1.0739219291010515e6 1.0734778851895705e6 1.0734125162961732e6 1.0729560849158617e6 1.0725485013548639e6 1.0723935441413159e6 1.072191473095047e6 1.0716950712528175e6 1.0717560127289307e6 1.0712823008363182e6 1.07126264085148e6 1.0709539070857568e6 1.0704259657617738e6 1.0700738464432743e6 1.0700970038982253e6; 1.1286328420708687e6 1.2845010258723828e6 1.09712619502174e6 1.1711324780941668e6 1.0743052119022994e6 1.0738981509752856e6 1.0738416252636781e6 1.0732036533027391e6 1.072909517672572e6 1.0729977726520041e6 1.072532510444904e6 1.0720619397527846e6 1.0716276998377366e6 1.07170647448822e6 1.0713438125759184e6 1.0712294869793372e6 1.0708237932263738e6 1.0703774141810236e6 1.0703500389779368e6 1.0701053080787896e6; 1.297697445935746e6 1.2985180008114118e6 1.2664523714107177e6 1.1208324319153856e6 1.073929431247634e6 1.0739029180960222e6 1.0734171945812833e6 1.0733887666948044e6 1.072886397439848e6 1.0729649575134106e6 1.0725726565010184e6 1.072542368171878e6 1.0716394994298525e6 1.0716773267408926e6 1.0712396958808545e6 1.0712823259616168e6 1.0707662299736987e6 1.070380947192292e6 1.0704972796297192e6 1.0701146050508523e6; 2.7539197062992514e6 2.735933828926478e6 1.3392654720119052e6 1.3257393723023129e6 1.284753939825337e6 1.0737858909678697e6 1.0733890081350815e6 1.073432232514114e6 1.0729434565854785e6 1.0730012550227896e6 1.0725581099038087e6 1.0720837225353569e6 1.0717200799732034e6 1.0717355325231152e6 1.071298508999703e6 1.0712037764696146e6 1.0708129280376774e6 1.070414113656442e6 1.0700379574417486e6 1.070023876473296e6; 2.3141586300850706e6 1.7963601194448366e6 1.8003091510687592e6 1.8140972140994598e6 1.1796422021399569e6 1.1583647151432177e6 1.0733857557524703e6 1.0734076356261105e6 1.073084043008801e6 1.0730140171549586e6 1.0725805136938419e6 1.072061471668085e6 1.0716988521680264e6 1.0717186889946212e6 1.071265269509978e6 1.0712138685983128e6 1.071028606985038e6 1.070472083999633e6 1.070097207230128e6 1.0700934140383273e6; 1.620898743566109e7 1.7979836624761468e6 1.7913041005017802e6 1.8106870235179872e6 1.3059940209902315e6 1.3058467065970297e6 1.0733790216077557e6 1.0734601720846936e6 1.0729563809923204e6 1.0726725886667392e6 1.0725266885772399e6 1.0721589570378447e6 1.0721022866621315e6 1.071658870813548e6 1.071308356634085e6 1.0708606635262645e6 1.0708990831201104e6 1.070857418634538e6 1.0703889014273195e6 1.0700221626895366e6; 1.4274997037597302e7 9.54404501941945e6 1.010361112930868e7 1.6478660345570054e6 1.608734450644874e6 1.2827154326628665e6 1.3053322593860503e6 1.072937812827555e6 1.0729910649208413e6 1.0725709077065713e6 1.0725393624321595e6 1.0720169901970609e6 1.0721034304954738e6 1.0714962981931355e6 1.071300900411566e6 1.0712733317291683e6 1.0708813514832077e6 1.0704933765678718e6 1.0703949160502148e6 1.0699881757533732e6; 8.755159910102213e6 8.762689105489377e6 5.5919386808439335e6 5.703876807359997e6 1.4468261407066165e6 1.334229127983581e6 1.3310529062931836e6 1.246412571974113e6 1.2348710120415147e6 1.0725360105990549e6 1.0724841108725101e6 1.0720833717746735e6 1.072017398634911e6 1.071241935207233e6 1.0712555434168146e6 1.0712320861904114e6 1.070971305951558e6 1.0704610555811701e6 1.0701057491628556e6 1.0699370103110075e6; 6.8474377396383e7 6.830085404498369e7 5.4426942452010205e6 5.105582467154619e6 3.553175322924828e6 3.4794501096286937e6 1.4095856312268332e6 1.3718913698171147e6 1.2908088112669452e6 1.2782354097918326e6 1.275483777985218e6 1.072066311548873e6 1.0720960696835967e6 1.0716366881624158e6 1.0716619498227723e6 1.0711933568825347e6 1.0708571142492208e6 1.0704878403116623e6 1.070029530944194e6 1.0700338645259307e6; 5.17710746477142e8 5.883113523995222e7 3.6625581314864695e7 3.758400251882986e7 2.7745618544309153e6 2.7750638006666754e6 2.826437965863329e6 2.064577802007899e6 1.278711063414084e6 1.0985953009198317e6 1.1092196926619154e6 1.2664773631122215e6 1.0721166947836482e6 1.071747283742483e6 1.0712474140107362e6 1.0710931258127152e6 1.0708719359072368e6 1.070503299687626e6 1.0700953382751849e6 1.070086202515147e6; 4.87261208961402e8 2.792812708396251e8 2.788310216697312e8 2.7957122407722896e8 2.040464100185768e7 2.002667580858441e7 2.069083610682468e6 2.116590509207684e6 1.3120958620566716e6 1.3129092703142914e6 1.3255290418092783e6 1.072265555591984e6 1.072148244070713e6 1.071710355799848e6 1.0716594917737688e6 1.0709107672563344e6 1.070881182778709e6 1.0704864864635896e6 1.070103190709334e6 1.070101397861547e6; 4.06957607533015e9 1.4677470116452897e8 1.3169456309324686e8 1.2795880620090482e8 1.2750119085227746e8 1.7882218217021473e7 1.1714265778245952e7 1.1857951787986154e7 2.1164915019879043e6 1.7838757063725728e6 1.7933559815982569e6 1.3098481455263603e6 1.085315412382539e6 1.071670872163898e6 1.071174685821165e6 1.0713190876639984e6 1.0707891187417223e6 1.0703961976308997e6 1.0703017112891888e6 1.0699684486622766e6; 1.966227068689923e9 1.9421464819857976e9 1.9347221297078676e9 6.640278523460366e7 6.3233420828927055e7 6.257081623718488e7 6.275781779768365e7 3.375566089288044e6 5.302941078171426e6 5.18141827591278e6 1.3543912617025515e6 1.2866955469210178e6 1.286698613780436e6 1.2735549496667683e6 1.2473961294849464e6 1.070865887632996e6 1.070892017125126e6 1.0705094353733826e6 1.0700055222352562e6 1.070019585664559e6; 1.8880108988586357e10 1.9009744633574356e10 9.409185884167161e8 9.365007595948614e8 9.445058958624914e8 6.1929326816666365e7 3.7028099498601384e7 3.518318578747906e6 3.48198790774161e6 3.353874030516305e6 3.4837018291889112e6 1.3578285820664563e6 1.3641053889614618e6 1.3587705702796788e6 1.1770854710075224e6 1.0712433824530947e6 1.0708138395845108e6 1.0703836287986275e6 1.0700993198038351e6 1.0700318696186324e6; 2.4985527069380505e11 8.72547124092596e9 8.76125015231099e9 2.227283644214833e8 4.7892399195788854e8 4.6985264795758915e8 3.514555300376823e7 3.391978134093889e7 3.450552762432236e7 3.377834060498591e7 2.0690332570866651e6 2.086760633875291e6 2.05694871408271e6 1.2712211983302052e6 1.1298015200542025e6 1.0949426278455772e6 1.0708090628158497e6 1.0704200635929669e6 1.0704211254369665e6 1.0696490621631036e6; 1.0209342998828879e11 1.6791972823736267e9 3.857151999919975e9 3.839034940733918e9 3.7408727691860914e9 2.184196499837208e8 2.1516399313054636e8 2.2494656116250035e8 1.9024153935273733e7 1.7840068985827275e7 1.730356257353383e7 1.662882303673733e6 1.6879601017006673e6 1.6176371581768987e6 1.2302734422082242e6 1.292646954453848e6 1.1602546463434205e6 1.267813573577134e6 1.0700188987203294e6 1.0696396700507903e6; 1.523553812519518e11 5.2202545986153e10 5.11485880814384e10 5.127179400403125e10 1.712469593980492e9 1.7028303605083323e9 1.65698812636014e9 5.676829090317101e7 9.647499835395667e7 1.0418578503893487e8 4.882417815919635e6 8.238164288415822e6 1.3093123634431812e6 1.310026780173516e6 1.3029681335082531e6 1.299288025623562e6 1.1831607720003983e6 1.2802542491414642e6 1.0704354589237943e6 1.0702774110667235e6; 6.2275022130894615e10 6.2281675986856514e10 2.103445319840573e10 2.1148069784013515e10 2.096323657960568e10 7.71989955764061e8 1.879331512255918e9 9.263231895898783e8 8.984840106704838e8 5.469006435227833e7 5.4838928883665875e7 5.462312662163593e7 4.789177659890377e6 4.873769273567911e6 4.788656079292776e6 1.3994566659776932e6 1.3669859580841474e6 1.2701086007926504e6 1.070117201092508e6 1.07016352139177e6];

In [ ]:
f2 = x-> exp(cos(8pi*x))
P2 = ExtensionFramePlatform(FourierPlatform(),0.0..0.5)

In [ ]:
δs2 = 10. .^ LinRange(-10,-3,20)
ϵs2 = 10. .^ LinRange(-10,-3,20)
ls2 = zeros(length(δs2),length(ϵs2))
ns2 = zeros(length(δs2),length(ϵs2))
# This computation takes a very long time. Consider it running on a computation server. (Or use the AZ version)
# for (i,δ) in enumerate(δs2)
#     for (j,ϵ) in enumerate(ϵs2)
#         F = Fun(f2,P2;threshold=ϵ,δ=δ,solverstyle=DirectStyle(),directsolver=pQR_solver,
#             criterion=FNAStyle(),verbose=false)
#         @show δ, ϵ, length(F), norm(coefficients(F))
#         ls2[i,j] = length(F)
#         ns2[i,j] = norm(coefficients(F))
#     end
# end

ls2 = [257.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 977.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 83.0 85.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 69.0 87.0 973.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 67.0 171.0 71.0 99.0 129.0 129.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 63.0 65.0 65.0 995.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 61.0 61.0 61.0 61.0 63.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 59.0 61.0 61.0 61.0 61.0 65.0 61.0 61.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 57.0 57.0 59.0 59.0 59.0 61.0 61.0 63.0 129.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 55.0 55.0 55.0 55.0 55.0 57.0 57.0 57.0 127.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 53.0 53.0 53.0 53.0 53.0 53.0 55.0 61.0 57.0 57.0 61.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 49.0 49.0 51.0 51.0 51.0 51.0 51.0 51.0 51.0 117.0 121.0 57.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 47.0 47.0 47.0 49.0 49.0 49.0 51.0 51.0 51.0 55.0 253.0 127.0 63.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 45.0 45.0 45.0 45.0 45.0 45.0 47.0 49.0 49.0 49.0 49.0 49.0 57.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 43.0 43.0 43.0 43.0 43.0 43.0 43.0 45.0 45.0 47.0 47.0 47.0 47.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0 4097.0; 39.0 41.0 41.0 41.0 43.0 41.0 43.0 43.0 43.0 43.0 43.0 43.0 45.0 1003.0 53.0 4097.0 4097.0 4097.0 4097.0 4097.0; 37.0 37.0 37.0 37.0 39.0 39.0 41.0 41.0 41.0 41.0 41.0 41.0 41.0 66.0 43.0 47.0 4097.0 4097.0 4097.0 4097.0; 33.0 33.0 35.0 35.0 35.0 37.0 37.0 37.0 39.0 39.0 39.0 39.0 39.0 39.0 41.0 111.0 129.0 4097.0 4097.0 4097.0; 31.0 31.0 31.0 33.0 33.0 33.0 33.0 35.0 35.0 35.0 37.0 37.0 37.0 37.0 37.0 37.0 51.0 47.0 4097.0 4097.0; 29.0 29.0 29.0 29.0 29.0 29.0 31.0 31.0 31.0 33.0 33.0 33.0 35.0 35.0 35.0 35.0 35.0 47.0 4097.0 4097.0]
ns2 = [1.1707573196694565 1.074823152004736 1.074711329749346 1.0743151641481803 1.0742478912780868 1.073856892556155 1.073865463195352 1.0734253517181507 1.0729766806079495 1.0725407118452304 1.0724725984457528 1.0720370356599165 1.0721126250119353 1.0716846166760317 1.071413941286586 1.071177674816393 1.0707673666874742 1.0706129125927406 1.070047356260821 1.0700666844414994; 1.0995540472182657 1.0747573996968904 1.0746834957188327 1.0742557406987094 1.0738096330921436 1.073846844995651 1.0733644780124996 1.0732811579926034 1.0729383747751908 1.0728822868767531 1.0725662269790817 1.0720654610707294 1.0721921194169977 1.07170998063218 1.0713196476872018 1.0710032881508498 1.0707987998473156 1.0704885637887132 1.0704290463045842 1.069646085340265; 1.291023305669735 1.282700895686422 1.0746294979214415 1.0742574207421818 1.0742812965058217 1.0738380490764896 1.0733395792922795 1.0733622559832516 1.0730366052789413 1.0725568227467954 1.0725899833177903 1.0721533245660826 1.0720563782590389 1.0716653414434103 1.0711742921858367 1.0712285169913753 1.0708364887675854 1.0708491706609917 1.070306765050779 1.0701384500221354; 1.2890971959232507 1.2766480640559326 1.0972607554398404 1.0742392620213754 1.0741588039730752 1.0738204723155211 1.0734170007980022 1.0733739594785296 1.072948767305204 1.0729560298522482 1.0724745695406754 1.0720476111460853 1.0717470023351068 1.0717589127479805 1.0713527678318981 1.0707688168641967 1.0707100757663408 1.0704385424862792 1.0700376107303846 1.0700520849608566; 1.2997533354051856 1.2157956767291587 1.3079723901183935 1.2606680170586149 1.2346786798407547 1.2195984826664574 1.073590654999248 1.0733596567715586 1.0730393024125862 1.0726460332992853 1.0725273003978315 1.0720270224269797 1.072120994310221 1.0717328906716503 1.0712329588638665 1.0711708991990094 1.0709013003354628 1.0704906372986518 1.0705186758522183 1.069997982600047; 2.746168169199202 1.3307848492903642 1.326459833235257 1.095429867547514 1.073809353479505 1.073940296870344 1.0738687924089814 1.0730311440195632 1.0729948269882976 1.0730304851048447 1.0725210966791117 1.0721037953363484 1.072087668755093 1.071682989506968 1.0711503952213404 1.0712033648391328 1.0712768891833193 1.0705928497228607 1.0701196760882716 1.070070750971371; 2.393869032711712 1.7906673728202498 1.7993145650661666 1.3075593251769244 1.2844568741641469 1.073988061442183 1.073365599247814 1.073422295857385 1.0729514160269693 1.0729909108942155 1.0724606940762924 1.0721443328068758 1.0716818609421996 1.0716316270473447 1.0712929723392086 1.0712823670263367 1.0708048176162652 1.0709066072240248 1.0703820362332708 1.0701248403404118; 15.841819041001106 2.3964965894920325 1.7954647538051134 1.3055188479200763 1.3057866830166516 1.284273023149167 1.3055130039917353 1.302409623279288 1.0730285191886009 1.0729546923197706 1.0725666580824362 1.0721608603423134 1.0721095192483006 1.0716277544031931 1.071316373740288 1.0713517164983863 1.0707490614275132 1.0708707913689413 1.0704960480783692 1.0701542559859487; 9.822570587538545 9.79626961853873 1.8212963728719154 1.841219304883561 1.6003622500438552 1.3027444868344975 1.3029823759142385 1.2846444482074317 1.2188756098392284 1.072604651914747 1.0724743878044765 1.0721723264585177 1.072173616958579 1.0717308542858344 1.071348412976771 1.0708123683948714 1.0708586451762783 1.070421758383495 1.070047620254184 1.0701445180098828; 8.170455133020116 8.252170417211518 8.228514143480332 5.62938780868957 5.68461693187646 1.3344291041540435 1.336268529797267 1.2905538849841716 1.2105181771554925 1.0726318167602118 1.072441848593997 1.0720678663434748 1.0717346906192908 1.0718279805771498 1.0713239395078662 1.0708289186335838 1.0709759266361643 1.0705907799674568 1.070070001955716 1.070160562110628; 69.33351798432881 68.55012886703732 69.33562634562396 4.890169378045764 3.45522775066457 3.557676099627144 1.3949098945137708 1.3027407525401895 1.2906777117771142 1.304758159856003 1.2780729197170093 1.0721725345032542 1.072118618515306 1.0716728963812456 1.0713001715946453 1.0712673301623508 1.070819632636202 1.0703812741516114 1.0704138675668469 1.0699338167791124; 522.9078090855927 34.41960614984044 37.66415429810417 37.07786755985154 2.884108544006896 3.1376143341043172 2.811467655016322 2.0662943867873387 2.1077118186006714 1.215801767227883 1.2119894663833397 1.2680163213373732 1.0721343922467639 1.0717689630900118 1.0712787827480872 1.0708620865359302 1.0709076750619693 1.0708366842852821 1.070100948144704 1.0701544134034466; 472.6395689698429 278.73374436800384 278.3358351539875 33.70430304880586 20.52084375700203 20.562817306977514 2.0706795438586356 2.087536044867068 1.3128032242239454 1.2780427826003655 1.1436533322256441 1.1962422991475703 1.2623471004116316 1.071762457476416 1.0713131647832699 1.0710949865374553 1.0708013504377814 1.070846021629764 1.070054205572834 1.070060626817202; 4020.372600284713 126.35264635394311 126.6302351513326 142.38655568977734 122.62114886432765 5.462186718816318 11.84128526345773 2.0969064857838653 1.8011557358549537 1.8126269432177047 1.2747289331158225 1.2776108557111425 1.2683787014245813 1.0716856026420019 1.0713148083490012 1.0711795474544428 1.070907971186432 1.0705169073563516 1.0703360386471805 1.0699438342080587; 1960.2436929694056 1936.0746157313033 1949.9812359513787 64.51927208009869 72.77246736704414 63.93504164264277 61.76572997495112 5.529334530048232 5.231391622101799 1.5124537311065485 1.353676880517347 1.3500260465492187 1.288761674718551 1.0716611171043198 1.0712867354098483 1.0710844359120293 1.0708288437681819 1.0702807409299888 1.0700391890576832 1.070053448086581; 18894.843546174252 945.4640910256793 943.4910575652742 933.0200832743371 68.11268236015317 34.624215134068024 60.99100759959589 3.3458288750062115 3.4259015880289225 3.65205430525346 3.295201015057889 1.2725850232603473 1.3552012666771895 1.0842383268672169 1.2827617449208788 1.071318574590742 1.0708375744042162 1.0705038814441679 1.0703949110952358 1.0700579705459263; 8938.022556774293 8797.737269463063 8727.640331744818 8738.596066006581 473.8712605085019 477.2261623285366 37.009434004103845 34.530414162523165 34.09048178661635 2.2188790121424353 2.3522572459734006 2.04061778589048 2.0512101878447413 1.2546195528124056 1.2711929172291163 1.255033507891321 1.070719722434621 1.0708444877162762 1.070482724627045 1.070048706365979; 103977.70441847338 102361.79452550739 3790.7067788711483 3764.782519864393 3711.030411718602 222.47509578024005 214.70445591959495 8.47815816020891 17.498606900654526 17.37246083610887 17.43830894094033 1.6275475689484074 1.693545164383159 1.6787827302600828 1.309821371666138 1.172274232296429 1.1672819144256832 1.07070253426688 1.069998255326478 1.070076463843905; 152746.49131974403 52807.64494793121 51266.99263794064 1968.5017641093202 1753.514347883841 1717.111236161222 1759.940362807913 104.20519337497743 100.68065532819703 101.4215751123282 8.029997646275353 8.21899554995989 1.3048830944746967 1.3221267271985095 1.30408830458722 1.298197906222507 1.2452862022289506 1.2546971157190372 1.0700943102723435 1.0701030964952574; 62140.37747025825 62428.32233601841 61985.44131798853 21179.10280790526 21016.340952931176 20972.81822568089 1903.6347790921907 923.7481251420777 921.7343020640025 54.97027150534549 55.21073983461534 54.68781670839522 4.901653261705954 4.81452893612336 4.769569801634931 1.3966959359859334 1.4000826484732272 1.2727471893925564 1.0700364305224146 1.0699783026429128];

In [ ]:
opts = @pgf {
        view = (0, 90),
        xlabel=L"\log_{10}(\delta)",
        ylabel=L"\log_{10}(\epsilon)",
        colorbar,
        "colormap/jet",
        width = ".25\\textwidth",
        scale_only_axis=true,
    }
p3opts = @pgf {surf,shader = "flat"}

In [ ]:
P = @pgf GroupPlot({opts...,group_style={
        x_descriptions_at="edge bottom",
        y_descriptions_at="edge left",
        horizontal_sep="5em",group_size="4 by 1",}},
    {title=L"$f_1, N_{\mathrm{opt}}$"},Plot3(p3opts,Table(log10.(ϵs), log10.(δs),log10.(ls2))),
    {title=L"$f_1, \|\mathbf x\|$"},Plot3(p3opts,Table(log10.(ϵs), log10.(δs),log10.(ns2))),
    {title=L"$f_2, N_{\mathrm{opt}}$"},Plot3(p3opts,Table(log10.(ϵs), log10.(δs),log10.(ls))),
    {title=L"$f_2, \|\mathbf x\|$"},Plot3(p3opts,Table(log10.(ϵs), log10.(δs),log10.(ns))),
)

In [ ]:
# DocumentPGFPlots.savefigs(joinpath(apath,"img","evsd"), P)